In [ ]:
import numpy as np
import pandas as pd
from astropy.stats import mad_std, sigma_clip
import re
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
from cytoolz import get_in
from astropy.time import Time
from datetime import datetime, timedelta
from pathlib import Path
import pickle

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.projects.sigma_circuits.segmentation as segmentation
import paulssonlab.projects.sigma_circuits.matriarch_stub as matriarch_stub
import paulssonlab.projects.sigma_circuits.experiment as experiment
import paulssonlab.io.metadata as metadata

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
hv.extension("bokeh")

# Load data

In [ ]:
# nd2_filename = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210324/PWM_flipped.nd2"
nd2_filename2 = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210204/psc101_bistable.nd2"

In [ ]:
nd2 = matriarch_stub.get_nd2_reader(nd2_filename)

In [ ]:
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210324flipped.pickle"
filename = (
    "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210204nonflipped_psc101.pickle"
)
data = pickle.load(open(filename, "rb"))
df = data["table"]
md = data["metadata"]

# Date

In [ ]:
md.keys()

In [ ]:
x = md["acquisition_times"]
(x[:100], x[-100:])

In [ ]:
import array

In [ ]:
24729414.28245997

In [ ]:
1.84467432e13

In [ ]:
label = "acquisition_times"
chunk_location = getattr(nd2.parser._label_map, label)
raw_data = nd2reader.common.read_chunk(nd2._fh, chunk_location)
raw_ary = np.array(array.array("d", raw_data))
(raw_ary[:20], raw_ary[-20:])

In [ ]:
nd2.parser._label_map.acquisition_times

In [ ]:
md

In [ ]:
md["acquisition_times"][:]

In [ ]:
x = md["acquisition_times"][:] != md["acquisition_times"][-1]

In [ ]:
sum(x)

In [ ]:
grid_df = experiment.get_grid(nd2)

In [ ]:
acq_times = np.fromiter(nd2._parser._raw_metadata.acquisition_times, dtype=np.float64)

In [ ]:
dtimeabsolute = get_in(
    (b"SLxPictureMetadata", b"dTimeAbsolute"),
    nd2._parser._raw_metadata.image_metadata_sequence,
)
dtimemsec = get_in(
    (b"SLxPictureMetadata", b"dTimeMSec"),
    nd2._parser._raw_metadata.image_metadata_sequence,
)
tdelta = timedelta(milliseconds=dtimemsec)

In [ ]:
tdelta.total_seconds()

In [ ]:
(acq_times[-1] - acq_times[0]) / 60 / 60

In [ ]:
x = np.diff(acq_times)
y = x[x < 100]

In [ ]:
x[5000]

In [ ]:
plt.hist(y, bins=50)

In [ ]:
len(acq_times)

In [ ]:
dtimeabsolute

In [ ]:
t = Time(dtimeabsolute, format="jd")

In [ ]:
t.to_datetime()

In [ ]:
dtimemsec

In [ ]:
nd2._parser._raw_metadata.acquisition_times??

In [ ]:
acq_times

In [ ]:
len(acq_times)

In [ ]:
df

In [ ]:
grid_df

In [ ]:
from itertools import product

In [ ]:
sizes = nd2.sizes

In [ ]:
rows, cols = np.meshgrid(np.arange(sizes["v"]), np.arange(sizes["t"]))

In [ ]:
pd.DataFrame({"pos": rows.ravel(), "t": cols.ravel(), "time": })

In [ ]:
# pd.DataFrame(ziplist(product(range(sizes["v"]), range(sizes["t"])))[-1])

In [ ]:
df

# Filtered

In [ ]:
%%time
cells = df.join(grid_df).reset_index()
filtered_cells = cells[
    cells["area"].between(300, 1500)
    & cells["GFP-PENTA"].between(100, 5000)
    & cells["row"].isin(["A", "C", "D", "E", "G", "H"])
][["GFP-PENTA", "row", "t"]]

filtered_cells2 = filtered_cells.groupby(["row", "t"]).apply(
    lambda y: sigma_clip(y, sigma=2, maxiters=10, masked=False)
)

means = filtered_cells2.apply(np.mean)
sigmas = filtered_cells2.apply(np.std)

limits = pd.DataFrame({"lower": means - sigmas, "upper": means + sigmas})
mean_plot = means.hvplot.line("t", by="row", logy=True)
noise_plot = limits.hvplot.area(
    x="t", y="lower", y2="upper", by="row", stacked=False, alpha=0.2, logy=True
)

# line_alpha = 0.2
# IPTG_starts = (
#     hv.VLine(12).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(60).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(108).opts(alpha=line_alpha, color="blue")
#     * hv.VLine(156).opts(alpha=line_alpha, color="blue")
# )
# EZ_starts = (
#     hv.VLine(30).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(78).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(126).opts(alpha=line_alpha, color="grey")
#     * hv.VLine(174).opts(alpha=line_alpha, color="grey")
# )
# aTc_starts = (
#     hv.VLine(42).opts(alpha=line_alpha, color="red")
#     * hv.VLine(90).opts(alpha=line_alpha, color="red")
#     * hv.VLine(138).opts(alpha=line_alpha, color="red")
#     * hv.VLine(186).opts(alpha=line_alpha, color="red")
# )

# (IPTG_starts*EZ_starts*aTc_starts*means*errors).opts(width=1000,height=400)
# (means * errors).opts(width=1000, height=400)

In [ ]:
(mean_plot * noise_plot).opts(width=800, height=400)